# 中間画像生成

In [127]:
import numpy
import cv2
import os
import glob
import pandas as pd
import shutil


◆処理概要  
・動画から30pと60pを作成  
・30pは既存  
・60pが30pを含んで作成  
  

【今回処理】
★60以下  
・（target -fps)/fpsの確率でコピーする  
60 -50/50 →1/5  
60-24/24 →1.5  
・nameはその名前-1とする  
gtはそのままコピーとする  

◆全て  
#【未作成】  
60pで不足しているGTは補完処理  
不足は60pと30pの比較  
globで取得し、pandas,concat  
今回処理の参考に  
・GTは下記の式で補完  
補完　1 2  4 → 2はbefore*abs(target-next)/(next -before) + next*abs(target-before)/(next-before)  
g1*abs(2-4) /(4-1)+ g4*abs(2-1)/(4-1)  
    g1(2/3 ) + g4 (1/3)  


# メイン処理

In [5]:
#csvリスト
path_get_dir = r"C:\Users\Ten\python\work\30_60transac\20200725"
csv_list = []
for curDir, dirs, files in os.walk(path_get_dir):
    for file in files:
        if file.endswith(".csv"):
            csv_list.append(os.path.join(curDir, file))

print(csv_list)           
#csvごとに更新
for csv in csv_list:
    fps = 40
    #新しく作成したimgのリスト
    img_new_name_list = []
    dir_name_csv = os.path.dirname(os.path.dirname(csv))
    gt_path = dir_name_csv+"\\gt\\"
    img_path = dir_name_csv+"\\img\\"
    gt_void_path = dir_name_csv+"\\gt\\Void\\"
    
    #処理①imgフォルダコピー    
    img_files = glob.glob(img_path+"*.jpg")
    img_num_list = [i for i in range(len(img_files))]
    img_num_list_temp = [i for i in range(10000)]

    #copy処理
    if fps>=60:
        pass
    else:
        copy_index=list(map(lambda x:round(x*1/((60-fps)/fps)), img_num_list_temp))
        #一致した項目だけコピーする、一致確認
        src_set = set(copy_index)
        tag_set = set(img_num_list)
        matched_list = list(src_set & tag_set)
        #copy実施
        for i in matched_list:
            name_temp = os.path.splitext(os.path.basename(img_files[i]))
            dir_name = os.path.dirname(img_files[i])
            copy_name = dir_name+"\\"+name_temp[0]+"_1"+name_temp[1]
            print("copy:{}".format(copy_name))
            shutil.copy2(img_files[i], copy_name)
    #再取得
    img_files = glob.glob(img_path+"*.jpg")

            
        
    #処理②voidフォルダコピー
    void_files = glob.glob(gt_void_path+"*.jpg")
    void_num_list = [i for i in range(len(void_files))]
    void_num_list_temp = [i for i in range(10000)]

    if fps>=60:
        pass
    else:
        copy_void_index=list(map(lambda x:round(x*1/((60-fps)/fps)), void_num_list_temp))
        #一致した項目だけコピーする、一致確認
        src_set = set(copy_void_index)
        tag_set = set(void_num_list)
        void_matched_list = list(src_set & tag_set)
        #copy実施
        for j in void_matched_list:
            name_temp = os.path.splitext(os.path.basename(void_files[j]))
            dir_name = os.path.dirname(void_files[j])
            void_copy_name = dir_name+"\\"+name_temp[0]+"_1"+name_temp[1]
            print("voidcopy:{}".format(void_copy_name))
            shutil.copy2(void_files[i], void_copy_name)
    #再取得
    void_files = glob.glob(gt_void_path+"*.jpg")
    #処理③gtのcsvファイルを増やす
    gt_read = pd.read_csv(csv)
    gt_read["temp"] = gt_read.index
    gt_read_temp = pd.DataFrame(img_files)
    gt_read_temp.columns = ["path"]
    temp = pd.merge(gt_read, gt_read_temp,on="path",how="right")
    temp = temp.sort_values("path")
    temp = temp.reset_index()
    temp = temp.fillna(method='ffill')
    #結合
    temp = temp.drop(["index", "temp"], axis=1)
    temp.to_csv(csv, index=False)

['C:\\Users\\Ten\\python\\work\\30_60transac\\20200725\\1\\gt\\1.csv']


In [ ]:
    #gt読み込み
    gt_read = pd.read_csv(csv)
    gt_read["temp"] = gt_read.index

    #中間生成対象行
    mid = ["H","W"]

    #flag行
    flag = ["occ", "frame"]

    #中間生成
    #gt_temp = gt_read[mid]-gt_read[mid].diff(periods=-1).applymap(lambda x:x/2)
    #gt_temp[mid] = gt_temp[mid].applymap('{:.2f}'.format)
    #flagはひとつ前をコピーする
    #gt_temp[flag] = gt_read[flag]
    #中間に挿入するための行
    #gt_temp["temp"] = gt_temp.index+0.5
    #ファイル名追加
    #gt_temp["path"] = pd.DataFrame(img_new_name_list)
    

    #結合
    #gt = pd.concat([gt_read,gt_temp]).sort_values("temp")
    #gt = gt.reset_index(drop=True)
    #gt = gt.drop("temp", axis=1)
    #gt = gt.dropna()
    #gt.to_csv(csv, index=False)


In [82]:
gt_read.path

0    C:\Users\Ten\python\work\30_60transac\temp\1\i...
1    C:\Users\Ten\python\work\30_60transac\temp\1\i...
2    C:\Users\Ten\python\work\30_60transac\temp\1\i...
3    C:\Users\Ten\python\work\30_60transac\temp\1\i...
4    C:\Users\Ten\python\work\30_60transac\temp\1\i...
Name: path, dtype: object

In [ ]:
    #処理③gtのcsvファイルを増やす
    
    #gt読み込み
    gt_read = pd.read_csv(csv)
    gt_read["temp"] = gt_read.index

    #中間生成対象行
    mid = ["H","W"]

    #flag行
    flag = ["occ", "frame"]

    #中間生成
    gt_temp = gt_read[mid]-gt_read[mid].diff(periods=-1).applymap(lambda x:x/2)
    gt_temp[mid] = gt_temp[mid].applymap('{:.2f}'.format)
    #flagはひとつ前をコピーする
    gt_temp[flag] = gt_read[flag]
    #中間に挿入するための行
    gt_temp["temp"] = gt_temp.index+0.5
    #ファイル名追加
    gt_temp["path"] = pd.DataFrame(img_new_name_list)
    

    #結合
    gt = pd.concat([gt_read,gt_temp]).sort_values("temp")
    gt = gt.reset_index(drop=True)
    gt = gt.drop("temp", axis=1)
    gt = gt.dropna()
    gt.to_csv(csv, index=False)


In [ ]:

#csvごとに更新
for csv in csv_list:
    #新しく作成したimgのリスト
    img_new_name_list = []
    dir_name_csv = os.path.dirname(os.path.dirname(csv))
    gt_path = dir_name_csv+"\\gt\\"
    img_path = dir_name_csv+"\\img\\"
    gt_void_path = dir_name_csv+"\\gt\\Void\\"
    
    #処理①imgフォルダを複数にする
    #元画像ディレクトリパス
    
    img_files = glob.glob(img_path+"*.jpg")
    for i in range(0,len(img_files)):
        try:
            i_file1 = img_files[i]
            i_file2 = img_files[i+1]
            name = mid_image(i_file1,i_file2,img_path)
            #listに格納
            img_new_name_list.append(name)
        except:
            pass
    
    #処理②voidフォルダを複数にする
    void_files = glob.glob(gt_void_path+"*.jpg")
    for j in range(0,len(void_files)):
        try:
            v_file1 = void_files[j]
            v_file2 = void_files[j+1]
            mid_image(v_file1,v_file2,gt_void_path)
        except:
            pass    
    
    #処理③gtのcsvファイルを増やす
    
    #gt読み込み
    gt_read = pd.read_csv(csv)
    gt_read["temp"] = gt_read.index

    #中間生成対象行
    mid = ["H","W"]

    #flag行
    flag = ["occ", "frame"]

    #中間生成
    gt_temp = gt_read[mid]-gt_read[mid].diff(periods=-1).applymap(lambda x:x/2)
    gt_temp[mid] = gt_temp[mid].applymap('{:.2f}'.format)
    #flagはひとつ前をコピーする
    gt_temp[flag] = gt_read[flag]
    #中間に挿入するための行
    gt_temp["temp"] = gt_temp.index+0.5
    #ファイル名追加
    gt_temp["path"] = pd.DataFrame(img_new_name_list)
    

    #結合
    gt = pd.concat([gt_read,gt_temp]).sort_values("temp")
    gt = gt.reset_index(drop=True)
    gt = gt.drop("temp", axis=1)
    gt = gt.dropna()
    gt.to_csv(csv, index=False)
    
    #print(dir_name_csv)
    

